# ACEpotentials.jl + AtomsBase.jl Tutorial

## Introduction

This tutorial demonstrates how the ACEpotentials.jl package
interoperates with the AtomsBase.jl ecosystem.

## Installation

add and load general packages used in this notebook.

In [1]:
using Pkg
# Uncomment the next line if installing Julia for the first time
# Pkg.Registry.add("General")
# Pkg.Registry.add
Pkg.add(["ExtXYZ", "Unitful", "Distributed", "AtomsCalculators",
         "Molly", "AtomsCalculatorsUtilities", "AtomsBuilder",
         "AtomsBase", "LinearAlgebra",
         ])

# ACEpotentials installation:
# If ACEpotentials has not been installed yet, uncomment the following lines
# Add the ACE registry, which stores the ACEpotential package information
Pkg.Registry.add(RegistrySpec(url="https://github.com/ACEsuit/ACEregistry"))
Pkg.add(["GeomOpt", ])
# Pkg.add(["ACEpotentials",])

    Updating registry at `~/.julia/registries/ACE`
    Updating git-repo `https://github.com/ACEsuit/ACEregistry`
    Updating registry at `~/.julia/registries/General.toml`
   Resolving package versions...
   Installed GR_jll ───────────── v0.73.16+0
   Installed Qt6ShaderTools_jll ─ v6.8.2+1
   Installed ThreadingUtilities ─ v0.5.4
   Installed OpenSSL ──────────── v1.5.0
   Installed ColorVectorSpace ─── v0.10.0
   Installed OrderedCollections ─ v1.8.1
   Installed TimerOutputs ─────── v0.5.29
   Installed Qt6Declarative_jll ─ v6.8.2+1
   Installed Qt6Wayland_jll ───── v6.8.2+0
   Installed Qt6Base_jll ──────── v6.8.2+1
   Installed GR ───────────────── v0.73.16
    Updating `~/ACEpotentials_v0_8/Project.toml`
⌅ [a963bdd2] + AtomsBase v0.4.2
⌃ [f5cc8831] + AtomsBuilder v0.2.1
⌃ [9855a07e] + AtomsCalculatorsUtilities v0.1.5
⌃ [aa0f7f06] + Molly v0.21.2
  [37e2e46d] + LinearAlgebra v1.11.0
    Updating `~/ACEpotentials_v0_8/Manifest.toml`
  [ec485272] + ArnoldiMethod v0.4.0
⌅ [a9b6321

   1497.0 ms  ✓ Transducers → TransducersReferenceablesExt
   2838.7 ms  ✓ UnitfulLatexify
   1163.8 ms  ✓ Transducers → TransducersAdaptExt
   3367.5 ms  ✓ StatsBase
   4267.7 ms  ✓ Qt6Declarative_jll
   2215.8 ms  ✓ StaticPolynomials
   6746.2 ms  ✓ Graphs
   2475.3 ms  ✓ DynamicPolynomials
    779.1 ms  ✓ Qt6Wayland_jll
   8467.3 ms  ✓ GPUCompiler
   5623.0 ms  ✓ FLoops
   4857.9 ms  ✓ Folds
   4471.3 ms  ✓ Optim
   7859.8 ms  ✓ ChainRules
    880.9 ms  ✓ ArrayInterface → ArrayInterfaceChainRulesExt
    988.6 ms  ✓ MLDataDevices → MLDataDevicesChainRulesExt
   6041.8 ms  ✓ Distributions
   1277.0 ms  ✓ Distributions → DistributionsChainRulesCoreExt
   1429.0 ms  ✓ Distributions → DistributionsTestExt
   1975.1 ms  ✓ MultivariateStats
  19308.0 ms  ✓ CellListMap
   2466.0 ms  ✓ KernelDensity
  12945.6 ms  ✓ PlotUtils
   5649.7 ms  ✓ PlotThemes
  25006.1 ms  ✓ HTTP
   7741.9 ms  ✓ RecipesPipeline
  28489.1 ms  ✓ PrettyTables
   5236.4 ms  ✓ GR
  36273.8 ms  ✓ LoopVectorization
   1085

  We can check the status of the installed packages.

In [2]:
using Pkg; Pkg.activate(".")
Pkg.status()

  Activating project at `~/ACEpotentials_v0_8`


Status `~/ACEpotentials_v0_8/Project.toml`
  [3b96b61c] ACEpotentials v0.8.6
⌅ [a963bdd2] AtomsBase v0.4.2
⌃ [f5cc8831] AtomsBuilder v0.2.1
  [a3e0e189] AtomsCalculators v0.2.3
⌃ [9855a07e] AtomsCalculatorsUtilities v0.1.5
⌃ [352459e4] ExtXYZ v0.2.1
  [ca147568] GeomOpt v0.1.0
  [7073ff75] IJulia v1.27.0
  [b964fa9f] LaTeXStrings v1.4.0
  [98b081ad] Literate v2.20.1
⌃ [aa0f7f06] Molly v0.21.2
  [6f286f6a] MultivariateStats v0.10.3
  [91a5bcdd] Plots v1.40.13
  [08abe8d2] PrettyTables v2.4.0
  [fd094767] Suppressor v0.2.8
  [1986cc42] Unitful v1.22.1
  [8ba89e20] Distributed v1.11.0
  [37e2e46d] LinearAlgebra v1.11.0
Info Packages marked with ⌃ and ⌅ have new versions available. Those with ⌃ may be upgradable, but those with ⌅ are restricted by compatibility constraints from upgrading. To see why use `status --outdated`


  Import all the packages that we will be using, create some processes
  for parallel model fitting

In [1]:
using ExtXYZ, Unitful, AtomsCalculators, Distributed, ACEpotentials,
      AtomsCalculatorsUtilities
using AtomsCalculatorsUtilities.SitePotentials: cutoff_radius
addprocs(10, exeflags="--project=$(Base.active_project())")
@everywhere using ACEpotentials

## Fit a potential for Cu

The tutorial can be adapted trivially to use datasets for Ni, Cu, Li, Mo, Si, Ge.

We generate a smallish model (about 300 basis functions) for Cu, using
correlation-order 3 (body-order 4), and default for rcut. Then we estimate
the model parameters using the standard BLR solver.

In [ ]:
@which acefit!(train, model;  solver=solver)
# Finding the smoothness prior construction
# @which ACEpotentials._make_prior(model, 4, nothing)
# @which ACEpotentials.Models.algebraic_smoothness_prior(model.model; p = 4)
# # ^^ makes smoothness prior in function just above - "P" is the regulariser, gamma (a
# diagonal matrix)


In [2]:
# generate a model for Cu
sym = :Cu
model = ace1_model(elements = [sym,], order = 3, totaldegree = [ 20, 16, 12 ])
@show length_basis(model)
@show cutoff_radius(model)
# estimate parameters
train, test, _ = ACEpotentials.example_dataset("Zuo20_$sym")
solver = ACEfit.BLR(; factorization = :svd)
acefit!(train, model;  solver=solver); GC.gc()
# quickly check test errors => 0.5 meV/atom and 0.014 eV/A are ok
ACEpotentials.compute_errors(test, model);

length_basis(model) = 319
cutoff_radius(model) = 6.5 Å
┌─────────┬──────────┬───────┬─────┬───────┬──────┐
│    Type │ #Configs │ #Envs │  #E │    #F │   #V │
├─────────┼──────────┼───────┼─────┼───────┼──────┤
│     nil │      262 │ 27416 │ 262 │ 82248 │    0 │
├─────────┼──────────┼───────┼─────┼───────┼──────┤
│   total │      262 │ 27416 │ 262 │ 82248 │    0 │
│ missing │        0 │     0 │   0 │     0 │ 1572 │
└─────────┴──────────┴───────┴─────┴───────┴──────┘


[ Info: Assembling linear problem.
[ Info:   - Creating feature matrix with size (82510, 319).
[ Info:   - Beginning assembly with processor count:  11.
Progress: 100%|█████████████████████████████████████████| Time: 0:01:42
[ Info:   - Assembly completed.
[ Info: Assembling full weight vector.
Progress: 100%|█████████████████████████████████████████| Time: 0:00:08
[ Info: Entering bayesian_linear_regression_svd
┌ Info: Computing SVD of (82510, 319) matrix
│   BLAS.get_num_threads() = 7
│   BLAS.get_config() =
│    LinearAlgebra.BLAS.LBTConfig
│    Libraries: 
└    └ [ILP64] libopenblas64_.so
[ Info: SVD completed after 0.03126132981666667 minutes
[ Info: Beginning to maximize marginal likelihood


Iter     Function value   Gradient norm 
     0     7.612258e+04     8.231737e+04
 * time: 0.032427072525024414
     1     1.658610e+04     1.693985e+05
 * time: 1.0040159225463867
     2    -2.020275e+05     2.126882e+06
 * time: 1.0045549869537354
     3    -2.319633e+05     1.246268e+06
 * time: 1.0053200721740723
     4    -2.329936e+05     9.076053e+05
 * time: 1.0057649612426758
     5    -2.333663e+05     4.700206e+04
 * time: 1.0061039924621582
     6    -2.333677e+05     1.073410e+04
 * time: 1.006500005722046
     7    -2.334444e+05     3.500871e+05
 * time: 1.0071709156036377
     8    -2.369560e+05     1.911014e+06
 * time: 1.0076749324798584
     9    -2.388221e+05     7.352762e+05
 * time: 1.008194923400879
    10    -2.397617e+05     2.722812e+05
 * time: 1.0085749626159668
    11    -2.418037e+05     8.928362e+05
 * time: 1.0090749263763428
    12    -2.421421e+05     4.636605e+04
 * time: 1.009671926498413
    13    -2.423753e+05     3.685487e+05
 * time: 1.01020908355

┌ Info: Optimization complete
│   Results =
│     * Status: success
│    
│     * Candidate solution
│        Final objective value:     -2.425201e+05
│    
│     * Found with
│        Algorithm:     L-BFGS
│    
│     * Convergence measures
│        |x - x'|               = 3.13e-13 ≤ 1.0e-08
│        |x - x'|/|x'|          = 6.55e-15 ≰ 0.0e+00
│        |f(x) - f(x')|         = 0.00e+00 ≤ 0.0e+00
│        |f(x) - f(x')|/|f(x')| = 0.00e+00 ≤ 0.0e+00
│        |g(x)|                 = 1.49e-09 ≰ 0.0e+00
│    
│     * Work counters
│        Seconds run:   1  (vs limit Inf)
│        Iterations:    23
│        f(x) calls:    113
│        ∇f(x) calls:   113
└    
[ Info: RMSE Table


┌──────┬─────────┬──────────┬─────────┐
│ Type │ E [meV] │ F [eV/A] │ V [meV] │
├──────┼─────────┼──────────┼─────────┤
│  nil │   0.502 │    0.014 │   0.000 │
├──────┼─────────┼──────────┼─────────┤
│  set │   0.502 │    0.014 │   0.000 │
└──────┴─────────┴──────────┴─────────┘
┌──────┬─────────┬──────────┬─────────┐
│ Type │ E [meV] │ F [eV/A] │ V [meV] │
├──────┼─────────┼──────────┼─────────┤
│  nil │   0.291 │    0.007 │   0.000 │
├──────┼─────────┼──────────┼─────────┤
│  set │   0.291 │    0.007 │   0.000 │
└──────┴─────────┴──────────┴─────────┘


[ Info: MAE Table


## Geometry Optimization with GeomOpt

( Note: we should use GeometryOptimization.jl, but this is not yet updated to
  AtomsBase.jl version 0.4. )

First import some stuff + a little hack to make GeomOpt play nice with
the latest AtomsBase. This is a shortcoming of DecoratedParticles.jl
and requires some updates to fully implement the AtomsBase interface.

In [5]:
using AtomsBuilder, GeomOpt, AtomsCalculators, AtomsBase
using AtomsBase: FlexibleSystem, FastSystem
using AtomsCalculators: potential_energy
function _flexiblesystem(sys)
   c3ll = cell(sys)
   particles = [ AtomsBase.Atom(species(sys, i), position(sys, i))
                 for i = 1:length(sys) ]
   return FlexibleSystem(particles, c3ll)
end;

We generate a cubic Cu unit cell, but our potential might not have the same
equilibrium bond distance as the default in AtomsBuilder, so we optimize
the unit cell.

In [ ]:
ucell = bulk(sym, cubic=true)
ucell, _ = GeomOpt.minimise(ucell, model; variablecell=true)

We keep the energy of the equilibrated unit cell to later compute the
defect formation energy.

In [ ]:
Eperat = potential_energy(ucell, model) / length(ucell)
@show Eperat;

Now that we have an equilibrated unit cell we enlarge it, and then delete
an atom to generate a vacancy defect.

In [ ]:
sys = _flexiblesystem(ucell) * (2,2,2)
deleteat!(sys, 1)
sys

Now we do another geometry optimization to get the equilibrium geometry.

In [ ]:
vacancy_equil, result = GeomOpt.minimise(sys, model; variablecell = false)
@show result.g_residual;

We get an estimate of the formation energy. Note this is likely a poor
estimate since we didn't train the model on vacancy configurations.

In [ ]:
E_def = potential_energy(vacancy_equil, model) - length(sys) * Eperat
@show E_def;

One of the most useful Julia packages is ForwardDiff. This is used in
AtomsCalculatorsUtilities to implement hessians. ACEpotentials.jl only
has to ensure that its site energies allow Dual numbers as inputs.

In [ ]:
using AtomsCalculatorsUtilities.SitePotentials: hessian
using LinearAlgebra: Symmetric, eigvals

H = ustrip.( hessian(vacancy_equil, model) )
Λ = eigvals(Symmetric(H))
S_vib = sum(λ -> λ > 1e-10 ? log(λ) : 0, Λ)

## Molecular Dynamics with Molly

In [ ]:
import Molly
sys = rattle!(bulk(sym, cubic=true) * (2,2,2), 0.03)
sys_md = Molly.System(sys; force_units=u"eV/Å", energy_units=u"eV")
temp = 298.0u"K"
sys_md = Molly.System(
   sys_md;
   general_inters = (model,),
   velocities = Molly.random_velocities(sys_md, temp),
   loggers=(temp = Molly.TemperatureLogger(100),
          energy = Molly.PotentialEnergyLogger(typeof(1.0u"eV"), 100),), )
simulator = Molly.VelocityVerlet(
   dt = 1.0u"fs",
   coupling = Molly.AndersenThermostat(temp, 3.0u"fs"), )

Molly.simulate!(sys_md, simulator, 1000)

# looks like a reasonably stable simulation.
@info("Temperature history:", sys_md.loggers.temp.history)
@info("Energy history:", sys_md.loggers.energy.history)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*